### Building Workflows and AI Agents through Multi-tool Collaboration

OpenBioMed provides a user-friendly interface for connnecting different models and tools to build your own workflow and AI agent. Before starting, we recommend reading our tutorials for [tools](./explore_ai4s_tools.ipynb) and [data structure](./manipulate_molecules.ipynb) to help you clarify potential misunderstandings and build high-quality workflows.

The workflow is typically configured with a yaml file composed of two parts: `tools` and `edges`. 

- The `tools` part is composed of a list of tools, where the user should cnofigure its name and inputs (if needed). Refer to `open_biomed/core/tool_registry.py` for our available tools and use the `print_usage()` function for configuring the required inputs of the tool. NOTE: If parts of the input takes the output of a prior tool, it will not be configured in this yaml file.
- The `edges` part is compose of a list of connections between tools. The output of the `start` node is passed to the inputs of the `end` node. Typically, the workflow can automatically determine the part of the input that the output should flow into (e.g. if the output of a tool is a `Molecule` object, it will be passed as the `molecule` input for a downstream tool). We also provide a way to explicitly specify the input field by adding a `name_mapping` to the edge.

### Demo Workflow
Here we explain the [demo workflow](../configs/workflow/demo.yaml):
```
tools:
  - name: molecule_name_request
    inputs:
      accession: dimethoxy-sulfanylidene-(3,5,6-trichloropyridin-2-yl)oxy-lambda5-phosphane
  - name: molecule_question_answering
    inputs:
      text: How can we group this molecule according to its molecular structure and functional groups?
edges:
  - start: 0
    end: 1
```
The workflow first obtains a molecule named 'dimethoxy-sulfanylidene-(3,5,6-trichloropyridin-2-yl)oxy-lambda5-phosphane' from PubChem (Tool No.0) and then asks a question of the molecule (Tool No.1). Tool No.0 takes an accession as input (specified in config) and generates a `Molecule` object. Tool No.1 takes a molecule (tool output) and a textual question (specified in config) as inputs and generates a textual answer. The output of `molecule_name_request` is automatically passed to the `molecule` input of `molecule_question_answering` (as the edge starts from Tool No.0 to Tool No.1).

Once the configuration file is specified, you can put it under `cnofigs/workflow` and simply execute the workflow by running: 
```
python open_biomed/core/workflow.py --workflow [YOUR_WORKFLOW_NAME]
```
or the following codes:

In [1]:
# Change working directory
import os
import sys
parent = os.path.dirname(os.path.abspath(''))
print(parent)
sys.path.append(parent)
os.chdir(parent)

import logging
logging.basicConfig(level=logging.ERROR)

/AIRvePFS/dair/luoyz-data/projects/OpenBioMed/OpenBioMed_arch


In [2]:
import asyncio
from open_biomed.core.workflow import Workflow
from open_biomed.utils.config import Config

workflow = Workflow(config=Config("./configs/workflow/demo.yaml"))
# Workflow exectution may contain asynchronous calls
asyncio.create_task(workflow.run(num_repeats=1))

/AIRvePFS/dair/conda_envs/biomed/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/AIRvePFS/dair/conda_envs/biomed/lib/python3.9/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Inference Steps: 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]

([<open_biomed.data.text.Text object at 0x7fe55ea3f070>], ['N-[[(3R,9R,10S)-12-[(2S)-1-hydroxypropan-2-yl]-16-(methanesulfonamido)-3,10-dimethyl-13-oxo-2,8-dioxa-12-azabicyclo[12. 4. 0]octadeca-1(14),15,17-trien-9-yl]methyl]-N-methyl-2-phenylacetamide'])


<Task pending name='Task-5' coro=<Workflow.run() running at /AIRvePFS/dair/luoyz-data/projects/OpenBioMed/OpenBioMed_arch/open_biomed/core/workflow.py:60>>

Now we have a workflow with the following tools: 
Tool No.1: PubChem query.
Inputs: {"accession": molecule name}
Outputs: A molecule from PubChem.


Tool No.2: Molecule question answering.
Inputs: {"molecule": a small molecule you are interested in. "text": a question about the molecule.}
Outputs: An answer to the question.


Repeating workflow execution No.1:
Next we execute Tool No.1.
 The inputs of this tool are: {"accession": "dimethoxy-sulfanylidene-(3,5,6-trichloropyridin-2-yl)oxy-lambda5-phosphane"}
./tmp/pubchem_query_pubchem_dimethoxy-sulfanylidene-(3,5,6-trichloropyridin-2-yl)oxy-lambda5-phosphane.sdf
The outputs of this tool are: {"molecule": "COP(=S)(OC)Oc1nc(Cl)c(Cl)cc1Cl"}


Tool No.1 passes its "molecule" output to Tool No.2
Next we execute Tool No.2.
 The inputs of this tool are: {"text": "How can we group this molecule according to its molecular structure and functional groups?", "molecule": "COP(=S)(OC)Oc1nc(Cl)c(Cl)cc1Cl"}


Inference Steps: 100%|██████████| 1/1 [00:00<00:00,  3.47it/s]


The outputs of this tool are: {"text": "organic phosphonate; organothiophosphate insecticide"}




### Drug Design Workflow
Here's a more [complicated workflow](../configs/workflow/stable_drug_design.yaml). It first visualizes a protein and a pocket within the protein, and generates a molecule that fits the pocket. Then, the workflow optimizes the molecule to reduce its liver toxcity. Finally, it visualizes the binding pose, reports the binding score, and estimates several properties of the molecule before and after optimization.

In [5]:
import asyncio
from open_biomed.core.workflow import Workflow
from open_biomed.utils.config import Config

workflow = Workflow(config=Config("./configs/workflow/stable_drug_design.yaml"))
# Workflow exectution may contain asynchronous calls
# The exectution of this workflow may take a bit longer
asyncio.create_task(workflow.run(num_repeats=1, context=open("./logs/workflow_outputs1.txt", "w")))

<Task pending name='Task-7' coro=<Workflow.run() running at /AIRvePFS/dair/luoyz-data/projects/OpenBioMed/OpenBioMed_arch/open_biomed/core/workflow.py:60>>

./tmp/protein_to_visualize.pdb [414, 415, 416, 417, 418, 292, 293, 294, 295, 424, 426, 302, 303, 304, 305, 427, 428, 429, 301, 313, 314, 315, 316, 317, 318, 329, 332, 333, 335, 336, 344, 345, 346, 347, 348, 349, 425, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 371, 372, 375, 358]
 PyMOL not running, entering library mode (experimental)
 Movie: frame    1 of   20, 5.47 sec. (0:01:49 - 0:01:49 to go).
 Movie: frame    2 of   20, 4.68 sec. (0:01:28 - 0:01:36 to go).
 Movie: frame    3 of   20, 4.37 sec. (0:01:18 - 0:01:27 to go).
 Movie: frame    4 of   20, 4.40 sec. (0:01:14 - 0:01:20 to go).
 Movie: frame    5 of   20, 4.25 sec. (0:01:07 - 0:01:14 to go).
 Movie: frame    6 of   20, 4.36 sec. (0:01:05 - 0:01:08 to go).
 Movie: frame    7 of   20, 4.51 sec. (0:01:03 - 0:01:04 to go).
 Movie: frame    8 of   20, 5.19 sec. (0:01:07 - 0:01:00 to go).
 Movie: frame    9 of   20, 5.18 sec. (0:01:02 - 0:00:56 to go).
 Movie: frame   10 of   20, 5.29 sec. (0:00:58 - 0:00:52 to go).
 Movie

Inference Steps: 100%|██████████| 1/1 [00:04<00:00,  4.38s/it]


./tmp/structure-based_drug_design_mol_1741161304846_0.sdf


Inference Steps: 100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


./tmp/text-based_molecule_editing_mol_1741161308373_0.sdf
 PyMOL not running, entering library mode (experimental)
 Movie: frame    1 of   20, 1.59 sec. (0:00:31 - 0:00:31 to go).
 Movie: frame    2 of   20, 1.61 sec. (0:00:30 - 0:00:30 to go).
 Movie: frame    3 of   20, 1.60 sec. (0:00:28 - 0:00:28 to go).
 Movie: frame    4 of   20, 1.55 sec. (0:00:26 - 0:00:26 to go).
 Movie: frame    5 of   20, 1.58 sec. (0:00:25 - 0:00:25 to go).
 Movie: frame    6 of   20, 1.66 sec. (0:00:24 - 0:00:23 to go).
 Movie: frame    7 of   20, 1.68 sec. (0:00:23 - 0:00:22 to go).
 Movie: frame    8 of   20, 1.71 sec. (0:00:22 - 0:00:21 to go).
 Movie: frame    9 of   20, 1.70 sec. (0:00:20 - 0:00:19 to go).
 Movie: frame   10 of   20, 1.77 sec. (0:00:19 - 0:00:18 to go).
 Movie: frame   11 of   20, 1.73 sec. (0:00:17 - 0:00:16 to go).
 Movie: frame   12 of   20, 1.73 sec. (0:00:15 - 0:00:14 to go).
 Movie: frame   13 of   20, 1.67 sec. (0:00:13 - 0:00:13 to go).
 Movie: frame   14 of   20, 1.60 sec. (0

Inference Steps: 100%|██████████| 1/1 [00:00<00:00, 245.37it/s]


### Trial-and-Errors with Workflow

Notably, some of the tools may generate multiple outputs, or generate different outputs in different runs. Due to our architectural design, we currently choose a random sample from the multi-outputs as the inputs for downstream tools. Fortunately, the `workflow.run()` takes a `num_repeats` argument to execute the workflow for multiple times, allowing a trail-and-error process. 

### Developing LLM Agent based on Workflow
Details of the workflow execution are passed into the `context` argument of the function, which could be used by an LLM agent to summarize and obtain insights within the process. 

In [ ]:
from open_biomed.core.llm_request import ReportGeneratorSBDD

# Initialize the agent
# A simple report structure is defined in ReportGeneratorGeneral for general tasks. To obtain insights for your specific workflow, we suggest to create the corresponding structure. ReportGeneratorSBDD is an example for drug design task.
agent = ReportGeneratorSBDD()

# Pass the details of the workflow execution as the context
context = agent._gen_context("./logs/workflow_outputs1.txt")

# Summarize and obtain insights
resp = agent.llm.generate(query="", context=context)

In [ ]:
print ("[Report]\n", resp['final_resp'])

In [ ]:
print ("\n[Thoughts]\n", resp['reasoning'])